In [1]:
import os
import glob
import xml.etree.ElementTree as ET
import cv2
from pathlib import Path

# CONFIGURAÇÕES (Ajuste se necessário)
PASTA_ENTRADA = "dataset_raw"  # Onde você colocou os XMLs e JPGs
PASTA_SAIDA = "datasets/park-eye/labels/all" # Onde vamos salvar os TXTs convertidos
CLASSE_ID = 0  # 0 será o ID para "veiculo" no YOLO

# Cria a pasta de saída se não existir
os.makedirs(PASTA_SAIDA, exist_ok=True)

def converter_xml_para_yolo(caminho_xml):
    # 1. Ler o arquivo XML
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # 2. Descobrir o nome da imagem correspondente
    # O dataset PKLot geralmente tem o mesmo nome para xml e jpg
    nome_xml = Path(caminho_xml).name
    nome_jpg = nome_xml.replace('.xml', '.jpg')
    caminho_imagem = os.path.join(PASTA_ENTRADA, nome_jpg)
    
    # Precisamos abrir a imagem para saber a altura e largura total (para normalizar)
    if not os.path.exists(caminho_imagem):
        print(f"ALERTA: Imagem {nome_jpg} não encontrada. Pulando.")
        return

    img = cv2.imread(caminho_imagem)
    altura_img, largura_img, _ = img.shape
    
    linhas_yolo = []

    # 3. Varrer cada vaga (<space>) dentro do XML
    for space in root.findall('space'):
        # Só queremos treinar o modelo para achar CARROS.
        # Então filtramos apenas as vagas OCUPADAS (occupied="1")
        if space.get('occupied') == '1':
            
            # Pegar os pontos do contorno
            contour = space.find('contour')
            pontos_x = [int(p.get('x')) for p in contour.findall('point')]
            pontos_y = [int(p.get('y')) for p in contour.findall('point')]

            # Converter Polígono -> Retângulo (Bounding Box)
            x_min, x_max = min(pontos_x), max(pontos_x)
            y_min, y_max = min(pontos_y), max(pontos_y)

            # Matemática do YOLO (Centro + Tamanho Normalizado)
            largura_box = x_max - x_min
            altura_box = y_max - y_min
            x_centro = x_min + (largura_box / 2)
            y_centro = y_min + (altura_box / 2)

            # Normalização (0 a 1)
            x_c_norm = x_centro / largura_img
            y_c_norm = y_centro / altura_img
            w_norm = largura_box / largura_img
            h_norm = altura_box / altura_img

            # Adiciona na lista: 0 0.543 0.321 ...
            linhas_yolo.append(f"{CLASSE_ID} {x_c_norm:.6f} {y_c_norm:.6f} {w_norm:.6f} {h_norm:.6f}")

    # 4. Salvar o arquivo .txt se houver carros
    if linhas_yolo:
        caminho_txt = os.path.join(PASTA_SAIDA, nome_xml.replace('.xml', '.txt'))
        with open(caminho_txt, "w") as f:
            f.write("\n".join(linhas_yolo))
        return True # Indica sucesso
    return False

# Executar a conversão em lote
xmls = glob.glob(os.path.join(PASTA_ENTRADA, "*.xml"))
print(f"Encontrados {len(xmls)} arquivos XML em '{PASTA_ENTRADA}'. Convertendo...")

convertidos = 0
for xml_file in xmls:
    if converter_xml_para_yolo(xml_file):
        convertidos += 1

print(f"Concluído! {convertidos} arquivos TXT gerados em '{PASTA_SAIDA}'.")

Encontrados 24 arquivos XML em 'dataset_raw'. Convertendo...
Concluído! 24 arquivos TXT gerados em 'datasets/park-eye/labels/all'.


In [2]:
import shutil
import random

# Definir caminhos finais
BASE_DIR = Path("datasets/park-eye")
IMG_DIR = BASE_DIR / "images"
LBL_DIR = BASE_DIR / "labels"

# Criar subpastas (train e val)
for tipo in ["train", "val"]:
    (IMG_DIR / tipo).mkdir(parents=True, exist_ok=True)
    (LBL_DIR / tipo).mkdir(parents=True, exist_ok=True)

# Listar todos os TXTs que criamos no passo anterior
txt_files = list((LBL_DIR / "all").glob("*.txt"))
random.shuffle(txt_files) # Embaralhar para garantir aleatoriedade

# Definir ponto de corte (80% treino, 20% validação)
split_idx = int(len(txt_files) * 0.8)
train_files = txt_files[:split_idx]
val_files = txt_files[split_idx:]

print(f"Separando: {len(train_files)} para Treino e {len(val_files)} para Validação...")

def mover_arquivos(lista_arquivos, destino_tipo):
    for txt_path in lista_arquivos:
        # 1. Mover o TXT
        shutil.copy(txt_path, LBL_DIR / destino_tipo / txt_path.name)
        
        # 2. Copiar a Imagem correspondente (que está na pasta raw)
        nome_jpg = txt_path.name.replace('.txt', '.jpg')
        src_jpg = Path(PASTA_ENTRADA) / nome_jpg
        if src_jpg.exists():
            shutil.copy(src_jpg, IMG_DIR / destino_tipo / nome_jpg)

# Executar a movimentação
mover_arquivos(train_files, "train")
mover_arquivos(val_files, "val")

print("Estrutura de pastas pronta para o YOLO!")

Separando: 19 para Treino e 5 para Validação...
Estrutura de pastas pronta para o YOLO!


In [3]:
from ultralytics import YOLO
import yaml

# 1. Criar o arquivo data.yaml
config_data = {
    'path': str(BASE_DIR.absolute()), # Caminho absoluto para evitar erros
    'train': 'images/train',
    'val': 'images/val',
    'names': {0: 'veiculo'} # Nome da nossa classe
}

with open("data.yaml", "w") as f:
    yaml.dump(config_data, f)

print("Arquivo data.yaml criado.")

# 2. Carregar e Treinar o Modelo
# Usamos o 'yolov8n.pt' como base (transfer learning)
print("Iniciando treinamento... (Isso pode demorar dependendo da sua GPU/CPU)")
model = YOLO("yolov8n.pt") 

# Treinar!
# epochs=50: Ele vai passar 50 vezes por todas as fotos
# imgsz=640: Tamanho da imagem que o YOLO vai 'enxergar'
results = model.train(data="data.yaml", epochs=50, imgsz=640)

print("Treinamento concluído!")

Arquivo data.yaml criado.
Iniciando treinamento... (Isso pode demorar dependendo da sua GPU/CPU)
Ultralytics 8.3.232  Python-3.13.7 torch-2.9.1+cpu CPU (Intel Core i5-9300H 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optim